In [24]:
import numpy as np
import matplotlib.pyplot as plt
import sys, os, cv2, random

caffe_root = '/home/heyanguang/caffecode/caffe/'
sys.path.insert(0, caffe_root + 'python')
os.chdir(caffe_root)
import caffe

sys.path.append('./examples/coco_caption')

iter_num = 110000
net = caffe.Net('./examples/coco_caption/lstm_lm.deploy.prototxt',
                './examples/coco_caption/lstm_lm_iter_%d.caffemodel' % iter_num, caffe.TEST)
print net.blobs['probs'].data.shape

(1, 1, 8801)


In [25]:
vocabulary = ['<EOS>'] + [line.strip() for line in
                          open('examples/coco_caption/h5_data/buffer_100/vocabulary.txt').readlines()]
print len(vocabulary)

8801


In [27]:
def predict_single_word(net, previous_word, output='probs'):
    cont = 0 if previous_word == 0 else 1
    cont_input = np.array([cont])
    word_input = np.array([previous_word])
    net.forward(cont_sentence = cont_input, input_sentence = word_input)
    output_preds = net.blobs[output].data[0, 0, :]
    return output_preds

In [29]:
# result = predict_single_word(net, 0)
# first_word_dist = result[0, 0, :]
first_word_dist = predict_single_word(net, 0)
print first_word_dist

[  1.76822894e-07   1.57942320e-03   6.74888968e-01 ...,   3.47725489e-07
   3.68430051e-07   7.07161462e-07]


In [30]:
# Arrange small real number first.
top_preds = np.argsort(-1 * first_word_dist)
print top_preds[:10]
print [vocabulary[index] for index in top_preds[:10]]

[  2  14   5  13  64  77  93  18  30 142]
['a', 'two', 'the', 'an', 'there', 'three', 'several', 'people', 'some', 'this']


In [32]:
second_word_dist = predict_single_word(net, vocabulary.index('two'))
print [vocabulary[index] for index in np.argsort(-1 * second_word_dist)[:10]]

['men', 'people', 'giraffes', 'women', 'zebras', 'cats', 'dogs', 'elephants', 'horses', 'girls']


In [35]:
third_word_dist = predict_single_word(net, vocabulary.index('giraffes'))
print [vocabulary[index] for index in np.argsort(-1 * second_word_dist)[:10]]

['and', 'in', 'standing', 'next', 'near', 'at', 'behind', 'one', 'inside', 'on']


In [59]:
third_word_dist = predict_single_word(net, vocabulary.index('eating'))
print [vocabulary[index] for index in np.argsort(-1 * second_word_dist)[:10]]

['from', 'grass', 'leaves', 'off', 'greens', 'and', ',', '<EOS>', 'hay', 'food']


In [61]:
def softmax(softmax_inputs, temp):
    shifted_inputs = softmax_inputs - softmax_inputs.max()
    exp_outputs = np.exp(temp * shifted_inputs)
    exp_outputs_sum = exp_outputs.sum()
    if np.isnan(exp_outputs_sum):
        return exp_outputs * float('nan')
    assert exp_outputs_sum > 0
    if np.isinf(exp_outputs_sum):
        return np.zeros_like(exp_outputs)
    eps_sum = 1e-20
    return exp_outputs / max(exp_outputs_sum, eps_sum)

def random_choice_from_probs(softmax_inputs, temp = 1):
    # temperature of infinity == take the max
    if temp == float('inf'):
        return np.argmax(softmax_inputs)
    probs = softmax(softmax_inputs, temp)
    r = random.random()
    cum_sum = 0.
    for i, p in enumerate(probs):
        cum_sum += p
        if cum_sum >= r: return i
    return 1  # return UNK?

In [184]:
def generate_sentence(net, temp = float('inf'), output = 'predict', max_words = 50):
    cont_input = np.array([0])
    word_input = np.array([1])
    sentence = []
    while len(sentence) < max_words and (not sentence or sentence[-1] != 0):
        net.forward(cont_sentence = cont_input, input_sentence = word_input)
        output_preds = net.blobs[output].data[0, 0, :]
        sentence.append(random_choice_from_probs(output_preds, temp = temp))
        cont_input[0] = 1
        word_input[0] = sentence[-1]
    return sentence

sentence = generate_sentence(net, temp = 10)
print sentence
print [vocabulary[index] for index in sentence]

[0]
['<EOS>']


In [190]:
sentence = generate_sentence(net, 10)
print sentence
print [vocabulary[index] for index in sentence]

[0]
['<EOS>']


In [105]:
np.array(vocabulary)[0:50]

array(['<EOS>', '<unk>', 'a', 'on', 'of', 'the', 'in', 'with', 'and', 'is',
       'man', 'to', 'sitting', 'an', 'two', ',', 'standing', 'at',
       'people', 'are', 'next', 'white', 'woman', 'table', 'street',
       'that', 'holding', 'it', 'person', 'large', 'some', 'down', 'top',
       'group', 'up', 'field', 'small', 'near', 'tennis', 'his', 'front',
       'black', 'plate', 'room', 'train', 'dog', 'riding', 'red', 'by',
       'young'],
      dtype='|S16')

In [135]:
sentence[-1]

0

In [103]:
!head examples/coco_caption/h5_data/buffer_100/vocabulary.txt

<unk>
a
on
of
the
in
with
and
is
man
